In [46]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix

import numpy as np
import pandas as pd
import os
import sys
sys.path.append('..')
import flood_tool as ft
import seaborn as sns

In [47]:
df = pd.read_csv(os.path.join(ft._data_dir,'postcodes_labelled.csv'))
df.drop(8713, inplace=True)
df.dropna(inplace=True)
df

,postcode,easting,northing,soilType,elevation,localAuthority,riskLabel,medianPrice,historicallyFlooded
0,OL9 7NS,390978,403269,Unsurveyed/Urban,130,Oldham,1,119100.0,False
1,WV13 2LR,396607,298083,Unsurveyed/Urban,130,Walsall,1,84200.0,False
2,LS12 1LZ,427859,432937,Unsurveyed/Urban,60,Leeds,1,134900.0,False
3,SK15 1TS,395560,397900,Unsurveyed/Urban,120,Tameside,1,170200.0,False
4,TS17 9NN,445771,515362,Unsurveyed/Urban,20,Stockton-on-Tees,1,190600.0,False
...,...,...,...,...,...,...,...,...,...
29993,NE65 8JL,409041,597599,Stagnosols,140,Northumberland,1,287100.0,False
29996,SK8 4PG,384808,387982,Unsurveyed/Urban,40,Stockport,1,328700.0,False
29997,HD7 4PA,409215,416819,Cambisols,310,Kirklees,1,214500.0,False
29998,NE16 5YT,419672,560517,Unsurveyed/Urban,130,Gateshead,1,273100.0,False


In [48]:
def add_postcode_features(df):
    listed_postcodes = df['postcode'].tolist()
    postcode_areas, postcode_districts, postcode_sectors, postcode_units = [], [], [], []

    for elem in listed_postcodes:

        if len(elem) == 7:
            postcode_area = elem[:2]
            postcode_district = elem[:3]
            postcode_sector = elem[:3]+elem[4:5]
            postcode_unit = elem[:3]+elem[4:7]

        if len(elem) == 6:
            postcode_area = elem[:1]
            postcode_district = elem[:2]
            postcode_sector = elem[:2]+elem[3:4]
            postcode_unit = elem[:2]+elem[3:6]
        
        if len(elem) == 8:
            postcode_area = elem[:2]
            postcode_district = elem[:4]
            postcode_sector = elem[:4]+elem[5:6]
            postcode_unit = elem[:4]+elem[5:8]

        postcode_areas.append(postcode_area)
        postcode_districts.append(postcode_district)
        postcode_sectors.append(postcode_sector)
        postcode_units.append(postcode_unit)
    df['postcode_area'] = postcode_areas
    df['postcode_district'] = postcode_districts
    df['postcode_sector'] = postcode_sectors
    df['postcode_unit'] = postcode_units
    return df

df = add_postcode_features(df)

In [49]:
# Preprocessing

def filter_by_percentile(group):
    lower = group['medianPrice'].quantile(0.1)
    upper = group['medianPrice'].quantile(0.9)
    return group[(group['medianPrice'] >= lower) & (group['medianPrice'] <= upper)]

def simplify_postcode(postcode):
    parts = postcode.split()
    if len(parts) > 1:
        return parts[0] + ' ' + parts[1][0]
    return parts[0]

def merging_dataframes(df1, df2, left_on, right_on, how='left'):
    merged = df1.merge(df2, left_on=left_on, right_on=right_on, how=how)
    return merged

def incomes_preprocessing(incomes):
    columns = incomes.iloc[0,:].values
    incomes.iloc[1:,:]
    incomes.columns = columns
    incomes = incomes.iloc[1:,:]
    incomes['Total annual income (£)'] = incomes['Total annual income (£)'].str.replace(',', '').astype(float)
    incomes['Total annual income (£)'].groupby(incomes['Local authority name']).mean()
    res = incomes['Total annual income (£)'].groupby(incomes['Local authority name']).mean()
    res = res.reset_index()
    return res

def modify_postcodeSector(postcode):
    postcode = postcode.replace(' ','')
    postcode = postcode.upper()
    return postcode

def preprocessing(df):
    sector_data = pd.read_csv(os.path.join(ft._data_dir,'sector_data.csv'))
    sector_data['postcodeSector'] = sector_data['postcodeSector'].apply(modify_postcodeSector)
    df = merging_dataframes(df, sector_data, left_on='postcode_sector', right_on='postcodeSector', how='left')

    return df

In [50]:
df = preprocessing(df)
df

,postcode,easting,northing,soilType,elevation,localAuthority,riskLabel,medianPrice,historicallyFlooded,postcode_area,postcode_district,postcode_sector,postcode_unit,postcodeSector,households,numberOfPostcodeUnits,headcount
0,OL9 7NS,390978,403269,Unsurveyed/Urban,130,Oldham,1,119100.0,False,OL,OL9,OL97,OL97NS,OL97,2538.0,260.0,8208.0
1,WV13 2LR,396607,298083,Unsurveyed/Urban,130,Walsall,1,84200.0,False,WV,WV13,WV132,WV132LR,WV132,2987.0,221.0,7031.0
2,LS12 1LZ,427859,432937,Unsurveyed/Urban,60,Leeds,1,134900.0,False,LS,LS12,LS121,LS121LZ,LS121,3016.0,277.0,5709.0
3,SK15 1TS,395560,397900,Unsurveyed/Urban,120,Tameside,1,170200.0,False,SK,SK15,SK151,SK151TS,SK151,4132.0,287.0,9519.0
4,TS17 9NN,445771,515362,Unsurveyed/Urban,20,Stockton-on-Tees,1,190600.0,False,TS,TS17,TS179,TS179NN,TS179,3386.0,210.0,7023.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26759,NE65 8JL,409041,597599,Stagnosols,140,Northumberland,1,287100.0,False,NE,NE65,NE658,NE658JL,NE658,1019.0,175.0,2631.0
26760,SK8 4PG,384808,387982,Unsurveyed/Urban,40,Stockport,1,328700.0,False,SK,SK8,SK84,SK84PG,SK84,3530.0,284.0,9028.0
26761,HD7 4PA,409215,416819,Cambisols,310,Kirklees,1,214500.0,False,HD,HD7,HD74,HD74PA,HD74,3724.0,239.0,7987.0
26762,NE16 5YT,419672,560517,Unsurveyed/Urban,130,Gateshead,1,273100.0,False,NE,NE16,NE165,NE165YT,NE165,4850.0,346.0,10309.0


In [51]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['medianPrice']), df['medianPrice'], test_size=0.2, random_state=42)

In [52]:
train = X_train.copy()
train['medianPrice'] = y_train

train = train.groupby('postcode_sector').apply(filter_by_percentile).reset_index(drop=True)

y_train = train['medianPrice']
X_train = train.drop(columns=['medianPrice'])

In [53]:
X_train.select_dtypes(exclude=np.number).columns.to_list()

['postcode',
 'soilType',
 'localAuthority',
 'historicallyFlooded',
 'postcode_area',
 'postcode_district',
 'postcode_sector',
 'postcode_unit',
 'postcodeSector']

In [54]:
X_train.select_dtypes(exclude=np.number).columns

Index(['postcode', 'soilType', 'localAuthority', 'historicallyFlooded',
       'postcode_area', 'postcode_district', 'postcode_sector',
       'postcode_unit', 'postcodeSector'],
      dtype='object')

In [55]:
num_features = ['easting', 'northing', 'elevation', 'households']
cat_features = ['postcode_district']
ord_features = ['soilType']

soilType_mapping = {'soilType': [
    'Luvisols',
    'Cambisols',
    'Arenosols',
    'Leptosols',
    'Podsols',
    'Planosols',
    'Stagnosols',
    'Gleysols',
    'Histosols',
    'Unsurveyed/Urban']}

In [56]:
class OrdinalTransformer(TransformerMixin, BaseEstimator):
    
    def __init__(self, category_mapping, unknown='ignore'):
        self.category_mapping = category_mapping
        self.unknown = unknown
        self.category_dicts = {col: {cat: idx for idx, cat in enumerate(categories)} for col, categories in category_mapping.items()}

    def fit(self, X=None, y=None):
        return self

    def transform(self, X, y=None):
        X = pd.DataFrame(X, columns=['soilType'])
        X_transformed = X.copy()
        for col, categories in self.category_mapping.items():
            X_transformed[col] = X[col].apply(lambda x: self.category_dicts[col].get(x, self.handle_unknown(col, x)))
        return X_transformed

    def handle_unknown(self, column, value):
        if self.unknown == 'ignore':
            return value
        elif self.unknown == 'use_max':
            return max(self.category_dicts[column].values()) + 1
        else:
            raise ValueError(f"Unknown handling mode '{self.unknown}' not supported.")

In [57]:
from sklearn.dummy import DummyRegressor

preprocessor = ColumnTransformer([
    ('num_transformer', make_pipeline(SimpleImputer(strategy='mean'),
                                              StandardScaler()), num_features),
    ('cat_transformer', make_pipeline(SimpleImputer(strategy ='most_frequent'), 
                                      OneHotEncoder(sparse_output=False, handle_unknown='ignore')), cat_features),
    ('ord_transformer', make_pipeline(SimpleImputer(strategy ='most_frequent'), OrdinalTransformer(soilType_mapping)), ord_features)
    ])

dummy_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', DummyRegressor(strategy='median'))])

dummy_pipeline.fit(X_train, y_train)
y_pred = dummy_pipeline.predict(X_test)

from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, y_pred, squared=False) #dummy (basic features) 928264.9956244312, 

928007.9858225469

In [58]:
preprocessor = ColumnTransformer([
    ('num_transformer', make_pipeline(SimpleImputer(strategy='mean'),
                                              StandardScaler()), num_features),
    ('cat_transformer', make_pipeline(SimpleImputer(strategy ='most_frequent'), 
                                      OneHotEncoder(sparse_output=False, handle_unknown='ignore')), cat_features),
    # ('ord_transformer', make_pipeline(SimpleImputer(strategy ='most_frequent'), 
    #                                    OrdinalTransformer(soilType_mapping)), ord_features)
    ])

rf_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('rf', RandomForestRegressor(random_state=42, n_estimators=100))
    ])

rf_pipeline.fit(X_train, y_train)
y_pred = rf_pipeline.predict(X_test)

from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, y_pred, squared=False) 

669040.661232275

# User Test Results

1. **Basic Features:** 775,318
2. **Features with Cats and Households:** 769,143.31
3. **Without Cats, with Households:** 770,339.19
4. **Without Soil Type:** 767,003.53
5. **Excluding Outliers (0.1, 0.9):** 865,672
6. **with filter :** 669,040